<a target="_blank" href="https://colab.research.google.com/github/novainsilico/jinko-api-cookbook/blob/main/tutorial/api_onboarding.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Module 6 : Leveraging Jinkō’s API for Programmatic Access

### Introduction
Goal of this cookbook is to illustrate how one can post the material needed for a trial, run it and visualize the results.

### Steps:
1. Load the Computational Model
2. Create a Virtual Population (Vpop) Design
3. Generate a Vpop from the Vpop Design
4. Post a Protocol
5. Post an Output set
6. Post a Trial
7. Run and monitor a Trial
8. Visualize the trial results
9. Hands-on!

### Linked resources:
- [API tutorials](https://doc.jinko.ai/docs/category/api---tutorials/): guidelines and examples on how to use the sdk
- [API Reference](https://doc.jinko.ai/api/#/): exhaustive documentation of jinkō's API
- [Public cookbook repository](https://github.com/novainsilico/jinko-api-cookbook/tree/main): cookbooks examples of use caes we compiled
- [Folder on jinko](https://jinko.ai/project/27e8eba0-a869-4c5d-b3b3-417db5a6e186): jinkō repository we're working with

---

## Step 0 : Connect to jinkō and setup the cookbook

libraries used in this notebook:

In [ ]:
import io
import json
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import sys
import os

install jinko-sdk

In [ ]:
!pip install jinko-sdk==0.5.0

connection initialization:

You need to create an api key [here](https://jinko.ai/project/27e8eba0-a869-4c5d-b3b3-417db5a6e186/settings) and to add it to your secrets.  
Project id can be retrieved in the url of the project

In [ ]:
import jinko_helpers as jinko

if "google.colab" in sys.modules:
    from google.colab import userdata
    JINKO_API_KEY = userdata.get('JINKO_API_KEY')
    print("JINKO_API_KEY is set")
else:
    JINKO_API_KEY = os.environ.get("JINKO_API_KEY")  # provide it

PROJECT_ID = "" # provide it

jinko.initialize(projectId=PROJECT_ID, apiKey=JINKO_API_KEY)

constants of the cookbook:

In [ ]:
# FOLDER_ID can be retrieved in the url of the folder, pattern is `https://jinko.ai/project/<project_id>?labels=<folder_id>`
FOLDER_ID = ""  # TO BE PROVIDED

# MODEL_URL is the url of your computational model, something like `https://jinko.ai/<model_sid>`
MODEL_URL = ""  # TO BE PROVIDED

# Step 1 : Load the Computational Model

In this step, we will use an already uploaded computational model,  
available at https://jinko.ai/cm-7etJ-gGTF

In [ ]:
# getting computational model information
model_project_item = jinko.get_project_item(url=MODEL_URL)

model_core_item_id = model_project_item["coreId"]["id"]
model_snapshot_id = model_project_item["coreId"]["snapshotId"]

display(model_project_item)

# Step 2 : Create a Vpop Design

Let's retrieve the patient descriptors from the model and define their distributions for the Vpop design creation.

In [ ]:
# retrieving baseline descriptors (https://doc.jinko.ai/api/#/paths/core-v2-model_manager-jinko_model-coreItemId--snapshots--snapshotId--baseline_descriptors/get)
numeric_descriptors = jinko.make_request(
    path=f"/core/v2/model_manager/jinko_model/{model_core_item_id}/snapshots/{model_snapshot_id}/baseline_descriptors",
).json()["numericDescriptors"]

patient_descriptors = [
    descriptor["id"]
    for descriptor in numeric_descriptors
    if any(
        tag in descriptor["inputTag"]
        for tag in [
            "PatientDescriptorKnown",
            "PatientDescriptorUnknown",
            "PatientDescriptorPartiallyKnown",
        ]
    )
]

print(f"{len(patient_descriptors)} patient descriptors:")
display(patient_descriptors)

Create a new list with the updated distributions:

In [ ]:
patient_descriptors_distribtion = [
    {
        "id": "Tu_init",
        "distribution": {"mean": 960000, "stdev": 1e5, "tag": "Normal"},
    },
    {
        "id": "mu2",
        "distribution": {"lowBound": 0.0328, "highBound": 0.0492, "tag": "Uniform"},
    },
    {
        "id": "p1",
        "distribution": {"lowBound": 1e-07, "highBound": 1.5e-07, "tag": "Uniform"},
    },
    {
        "id": "p2",
        "distribution": {"lowBound": 2.28e-07, "highBound": 3.42e-07, "tag": "Uniform"},
    },
    {
        "id": "p3",
        "distribution": {"lowBound": 8.8e-08, "highBound": 1.32e-07, "tag": "Uniform"},
    },
    {
        "id": "p4",
        "distribution": {"lowBound": 9.6e-09, "highBound": 1.44e-08, "tag": "Uniform"},
    },
    {
        "id": "p5",
        "distribution": {"lowBound": 2.4e-10, "highBound": 3.6e-10, "tag": "Uniform"},
    },
    {
        "id": "r",
        "distribution": {"lowBound": 0.00544, "highBound": 0.00816, "tag": "Uniform"},
    },
]

post the virtual population design:

In [ ]:
# uploading the vpop design (https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator/post)
response = jinko.make_request(
    path="/core/v2/vpop_manager/vpop_generator",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "marginalDistributions": patient_descriptors_distribtion,
        },
        "tag": "VpopGeneratorFromDesign",
    },
    options={
        "name": "vpop design for BCG model",
        "folder_id": FOLDER_ID,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
vpop_generator_core_item_id = project_item_info["coreItemId"]["id"]
vpop_generator_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(
    f"Virtual Populate Design Resource link: {jinko.get_project_item_url_from_response(response)}"
)

# Step 3 : Generate a Vpop from the Vpop design

From the virtual population design, we generate a virtual population of 100 patients

In [ ]:
# generating the vpop (https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator-coreItemId--snapshots--snapshotId--vpop/post)
response = jinko.make_request(
    path=f"/core/v2/vpop_manager/vpop_generator/{vpop_generator_core_item_id}/snapshots/{vpop_generator_snapshot_id}/vpop",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "size": 100,
        },
        "tag": "VpopGeneratorOptionsForVpopDesign",
    },
    options={
        "name": "vpop for BCG model",
        "folder_id": FOLDER_ID,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
vpop_core_item_id = project_item_info["coreItemId"]["id"]
vpop_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(
    f"Virtual Population Resource link: {jinko.get_project_item_url_from_response(response)}"
)

# Step 4 : Post a Protocol

We are posting a protocol with 2 arms:
 - Default_arm
 - Double_Dose

In [ ]:
scenario_arms = [
    {
        "armControl": None,
        "armIsActive": True,
        "armName": "Default_arm",
        "armOverrides": [
            {"formula": "100000.0", "key": "TreatmentBCG_Dose"},
            {"formula": "100.0", "key": "TreatmentBCG_NbDose"},
            {"formula": "7.0", "key": "TreatmentBCG_Recurrence"},
        ],
        "armWeight": 1,
    },
    {
        "armControl": None,
        "armIsActive": True,
        "armName": "Double_Dose",
        "armOverrides": [
            {"formula": "200000.0", "key": "TreatmentBCG_Dose"},
            {"formula": "100", "key": "TreatmentBCG_NbDose"},
            {"formula": "7", "key": "TreatmentBCG_Recurrence"},
        ],
        "armWeight": 1,
    },
]

In [ ]:
# uploading the protocol (https://doc.jinko.ai/api/#/paths/core-v2-scenario_manager-protocol_design/post)
response = jinko.make_request(
    path="/core/v2/scenario_manager/protocol_design",
    method="POST",
    json={"scenarioArms": scenario_arms},
    options={
        "name": "protocol for BCG model",
        "folder_id": FOLDER_ID,  # Post the protocol
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
protocol_core_item_id = project_item_info["coreItemId"]["id"]
protocol_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Protocol Resource link: {jinko.get_project_item_url_from_response(response)}")

# Step 5 : Post a Measure Design

We define which time series we want to observe in the simulation

In [ ]:
measures = [
    {"timeseriesId": "E"},
    {"timeseriesId": "T"},
    {"timeseriesId": "B"},
    {"timeseriesId": "Ti"},
]

In [ ]:
# uploading the measure design (https://doc.jinko.ai/api/#/paths/core-v2-scorings_manager-measure_design/post)
response = jinko.make_request(
    path="/core/v2/scorings_manager/measure_design",
    method="POST",
    json={
        "computationalModelId": {
            "coreItemId": model_core_item_id,
            "snapshotId": model_snapshot_id,
        },
        "measures": measures,
    },
    options={
        "name": "measure design for BCG model",
        "folder_id": FOLDER_ID,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
measure_design_core_item_id = project_item_info["coreItemId"]["id"]
measure_design_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(
    f"Measure Design Resource link: {jinko.get_project_item_url_from_response(response)}"
)

# Step 6 : Post a Trial

In [ ]:
# uploading the trial (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial/post)
response = jinko.make_request(
    path="/core/v2/trial_manager/trial",
    method="POST",
    json={
        "computationalModelId": {
            "coreItemId": model_core_item_id,
            "snapshotId": model_snapshot_id,
        },
        "protocolDesignId": {
            "coreItemId": protocol_core_item_id,
            "snapshotId": protocol_snapshot_id,
        },
        "vpopId": {"coreItemId": vpop_core_item_id, "snapshotId": vpop_snapshot_id},
        "measureDesignId": {
            "coreItemId": measure_design_core_item_id,
            "snapshotId": measure_design_snapshot_id,
        },
    },
    options={
        "name": "trial for BCG model",
        "folder_id": FOLDER_ID,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
trial_core_item_id = project_item_info["coreItemId"]["id"]
trial_snapshot_id = project_item_info["coreItemId"]["snapshotId"]

print(f"Trial Resource link: {jinko.get_project_item_url_from_response(response)}")

# Step 7 : Run and monitor a trial

We are running the trial simulation

In [ ]:
# running the trial (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-coreItemId--snapshots--snapshotId--run/post)
response = jinko.make_request(
    path=f"/core/v2/trial_manager/trial/{trial_core_item_id}/snapshots/{trial_snapshot_id}/run",
    method="POST",
)

getting the trial status


In [ ]:
# monitoring the trial status (https://doc.jinko.ai/docs/Tutorials/run-a-trial#monitoring-trial-status)
jinko.monitor_trial_until_completion(trial_core_item_id, trial_snapshot_id)

# Step 8 : Visualize the trial results

Once the trial completed, we can download and visualize the results

In [ ]:
# retrieving the output ids (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-coreItemId--snapshots--snapshotId--output_ids/get)
response = jinko.make_request(
    "/core/v2/trial_manager/trial/%s/snapshots/%s/output_ids"
    % (trial_core_item_id, trial_snapshot_id),
    method="GET",
)
responseSummary = json.loads(response.content.decode("utf-8"))
display("Available time series:", responseSummary)

download the time series:

In [ ]:
TIME_SERIES_IDS = ["B"]

try:
    print("Retrieving time series data...")
    # time series download (https://doc.jinko.ai/api/#/paths/core-v2-result_manager-timeseries_summary/post)
    response = jinko.make_request(
        "/core/v2/result_manager/timeseries_summary",
        method="POST",
        json={
            "select": TIME_SERIES_IDS,
            "trialId": {
                "coreItemId": trial_core_item_id,
                "snapshotId": trial_snapshot_id,
            },
        },
    )
    if response.status_code == 200:
        print("Time series data retrieved successfully.")
        archive = zipfile.ZipFile(io.BytesIO(response.content))
        filename = archive.namelist()[0]
        print(f"Extracted time series file: {filename}")
        csvTimeSeries = archive.read(filename).decode("utf-8")
    else:
        print(
            f"Failed to retrieve time series data: {response.status_code} - {response.reason}"
        )
        response.raise_for_status()
except Exception as e:
    print(f"Error during time series retrieval or processing: {e}")
    raise

load the time series in a dataframe:

In [ ]:
dfTimeSeries = pd.read_csv(io.StringIO(csvTimeSeries))
display(dfTimeSeries.head(5))

unique_patient_ids = dfTimeSeries["Patient Id"].unique().tolist()

plot the time series of the first patient:

In [ ]:
patient_data = dfTimeSeries[dfTimeSeries["Patient Id"] == unique_patient_ids[0]]

# Plot each arm separately
for arm, group in patient_data.groupby("Arm"):
    plt.plot(group["Time"], group["Value"], marker="o", linestyle="-", label=arm)

# Customize the plot
plt.title("Time Series of E")
plt.xlabel("Time (seconds)")
plt.ylabel("E")
plt.legend(title="Arm")

plt.show()

# Hands On part:

 - create a protocol with 50 arms where Treatment BCG Dose varies from 1e5 to 1e6
 - post a new trial with this protocol
 - run the trial
 - plot the time series

 Have fun :)